In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

In [2]:
dataset_names = [
         'adult',
         'Amazon_employee_access',
         'bank-marketing',
         'Click_prediction_small',
         'credit-g',
         ]

In [3]:
auto_ml_names = [
    'Auto_ml', 
    'h2o', 
    'tpot', 
    'catboost-default',
    'lightgbm-default',
    'lightgbm-opt',
    ]

In [4]:
METRIC = 'AUC'
metrics = ['AUC', 'log_loss', 'Accuracy']

In [5]:
bench_result = {}
for name in auto_ml_names:
    bench_result[name] = 0

In [26]:
total_place = pd.DataFrame()
total_score = pd.DataFrame()
total_score_std = pd.DataFrame()
#for metric in metrics:
metric = 'AUC'
bench_result = {}

for name in auto_ml_names:
    bench_result[name] = 0
for dataset_name in dataset_names:
    print(dataset_name)
    bench_auc = []
    bench_auc_std = []
    for AUTOML in auto_ml_names:
        result = pd.read_csv(f'./frameworks/{AUTOML}/result/{dataset_name}_{AUTOML}_metrics.csv')
        bench_auc.append(result[metric].mean())
        bench_auc_std.append(result[metric].std())

    res = pd.DataFrame(bench_auc, columns=[metric,], index=auto_ml_names).sort_values(by=[metric])
    res_std = pd.DataFrame(bench_auc_std, columns=[metric+'_std',], index=auto_ml_names)
    for i, name in enumerate(res.index):
        bench_result[res.index[i]] += i+1
#total_place.append(bench_result)
        bench = pd.io.json.json_normalize(bench_result,).T
        bench.columns = [f'{metric} place']
    res.columns=[dataset_name,]
    total_score = pd.concat([total_score, res], axis=1, sort=False)
    total_score = pd.concat([total_score, res_std], axis=1, sort=False)

    
total_place = pd.concat([total_place, bench], axis=1, sort=False)
        #bench = bench.sort_values(by=f'{metric} place', ascending=False)
        #bench.to_csv(f'{metric}_place.csv',)
#result.head(5)

adult
Amazon_employee_access
bank-marketing
Click_prediction_small
credit-g


In [27]:
total_score

,adult,AUC_std,Amazon_employee_access,AUC_std,bank-marketing,AUC_std,Click_prediction_small,AUC_std,credit-g,AUC_std
tpot,0.91260,0.002619,0.78954,0.033911,0.84918,0.007032,0.71136,0.004526,0.78164,0.018928
h2o,0.91428,0.002050,0.85510,0.002957,0.93710,0.003676,0.72056,0.004140,0.77646,0.047901
lightgbm-default,0.91444,0.003686,0.84626,0.011325,0.93646,0.003448,0.71598,0.005708,0.77954,0.027359
Auto_ml,0.91466,0.003276,0.82860,0.014350,0.90354,0.005755,0.71884,0.006636,0.79252,0.022704
lightgbm-opt,0.91482,0.003590,0.85770,0.008034,0.93848,0.003045,0.71728,0.004390,0.78524,0.031052
catboost-default,0.91498,0.002952,0.84666,0.009016,0.93792,0.004034,0.71912,0.005836,0.78374,0.022156


In [28]:
total_score.to_csv('total_score.csv')

In [25]:
total_score_std.to_csv('total_score_std.csv')

In [13]:
total_place

,AUC place
Auto_ml,18
h2o,18
tpot,7
catboost-default,24
lightgbm-default,13
lightgbm-opt,25


In [14]:
res

,AUC
h2o,0.77646
lightgbm-default,0.77954
tpot,0.78164
catboost-default,0.78374
lightgbm-opt,0.78524
Auto_ml,0.79252


In [16]:
bench_result

{'Auto_ml': 18,
 'h2o': 18,
 'tpot': 7,
 'catboost-default': 24,
 'lightgbm-default': 13,
 'lightgbm-opt': 25}

In [17]:
bench = pd.io.json.json_normalize(bench_result,).T

In [18]:
bench.columns = ['AUC place']

In [19]:
bench = bench.sort_values(by='AUC place', ascending=False)

In [44]:
# import matplotlib.pyplot as plt
# plt.style.use('bmh')
# #import seaborn as sns
# #sns.set_style(style="darkgrid")

In [43]:
#plt.rcdefaults()
# fig, ax = plt.subplots()

# colors = color=['tab:blue', 'firebric',]

# ax.barh(bench.index, bench['AUC place'], align='center', color=colors,)
# ax.invert_yaxis()
# plt.show()

In [45]:
bench.to_csv('auc_place', index=False,)

,AUC place
lightgbm-opt,25
catboost-default,24
Auto_ml,18
h2o,18
lightgbm-default,13
tpot,7
